# Data to ASDF

Since the downloaded data can't simply be used in the `.mseed` format,
the data needs to be converted into a more versatile format. This can 
be done using obspy and pyasdf.

## Import data as streams

In [1]:
import numpy
import pyasdf
from obspy import read, read_inventory
from gcmt3d import CMTSource

# Import event
cmt = CMTSource.from_CMTSOLUTION_file("data/eq_010202E/fijiCMT.cmt")
event_coordinates = (cmt.latitude,cmt.longitude)

# Read Station XML as inventory
inv = read_inventory("data/eq_010202E/station.xml")

# Import traces
stream = read("data/eq_010202E/seismograms/010202E.mseed")

# Remove the responses from the traces using the inventory
stream.remove_response(inventory=inv)

# Detrend the traces linearly
stream.detrend()

# Bandpass filter the traces using a butterworth filter
stream.filter("bandpass",freqmin=0.01, freqmax=1.5)


FileNotFoundError: [Errno 2] No such file or directory: 'data/eq_010202E/fijiCMT.cmt'

Now, what we have is a source, an inventory with the station information 
and a stream object containing traces, from which the response has been 
removed and which have been normalized. To get an overview of the recorded
traces the next step is to plot them in a section as a function of distance 
to the earthquake. To do this we need to add either the epicentral distance 
to `stream.trace.stats.distance` or we can add the location to 
`stream.trace.stats.coordinates.latitude` and 
`stream.trace.stats.coordinates.longitude`. This is done by looping through 
the traces and finding the necessary info in the inventory file:


In [ ]:
# Adding location to the traces
for st in stream:
    
    # Getting the seed_id to request the specific station location from inventory
    seed_id = st.stats.network + "." + st.stats.station + "." + st.stats.location + "." + st.stats.channel
    
    # Get station coordinates from inventory
    temp = inv.get_coordinates(seed_id)
    
    # Add coordinates to inventory
    st.stats.coordinates = {"latitude": temp["latitude"],
                            "longitude": temp["longitude"]}
    
#     st.stats.coordinates.latitude = temp["latitude"]
#     st.stats.coordinates.longitude = temp["longitude"]
    
    # 
    #     print(st.stats.coordinates.latitude)
    #     print(st.stats.coordinates.longitude)
 

In [ ]:
print(stream[0].stats.coordinates.latitude)


After adding the coordinates to the traces, we can plot them:

In [ ]:
print("Lat/Lon of first trace")
print(stream[0].stats.coordinates.latitude)
print(stream[0].stats.coordinates.longitude)
print("Lat/Lon of event")
print(event_coordinates[0])
print(event_coordinates[1])



In [ ]:
%matplotlib notebook
stream.plot(ev_coord=event_coordinates,type='section',dist_degree=True)

In [ ]:
print(stream[0].stats)


## Getting arrival times using the obspy built in TauP package

In [ ]:
from obspy import taup


arrivals = model.get_travel_times(source_depth_in_km=55,
                                  distance_in_degree=67)